Newer Version

In [77]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import precision_score, recall_score, f1_score

# ----- Step 1: Load and Prepare Weekly Data -----
df_weekly = pd.read_csv("/content/weekly_train.csv")
df_weekly['Week'] = pd.to_datetime(df_weekly['week'])  # Convert 'week' column to datetime
df_weekly = df_weekly.sort_values(by=['fdr_Id', 'Week'])

# Shift the Fault_Occurred column to create target labels
df_weekly['target'] = df_weekly.groupby('fdr_Id')['Fault_Occurred'].shift(-1)
df_weekly = df_weekly.dropna(subset=['target'])
df_weekly['target'] = df_weekly['target'].astype(int)

# ----- Step 2: Define Features and Normalize -----
exclude_cols = ['fdr_Id', 'Week', 'Fault_Occurred', 'target', 'week']
features = [col for col in df_weekly.columns if col not in exclude_cols]
scaler = MinMaxScaler()
df_weekly[features] = scaler.fit_transform(df_weekly[features])

# ----- Step 3: Create Feeder-Wise Sequences -----
def create_sequences_by_feeder(df, seq_length):
    X_list, y_list = [], []
    feeders = df['fdr_Id'].unique()
    for feeder in feeders:
        feeder_df = df[df['fdr_Id'] == feeder].reset_index(drop=True)
        for i in range(len(feeder_df) - seq_length):
            seq_X = feeder_df[features].iloc[i:i+seq_length].values
            seq_y = feeder_df['target'].iloc[i + seq_length]
            X_list.append(seq_X)
            y_list.append(seq_y)
    return torch.tensor(np.array(X_list), dtype=torch.float32), torch.tensor(np.array(y_list), dtype=torch.float32)

seq_short = 5
seq_long = 45

X_short, y_short = create_sequences_by_feeder(df_weekly, seq_short)
X_long, y_long = create_sequences_by_feeder(df_weekly, seq_long)

# Make sure the datasets are aligned in size
min_len = min(len(X_short), len(X_long))
X_short, y_short = X_short[:min_len], y_short[:min_len]
X_long, y_long = X_long[:min_len], y_long[:min_len]

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
X_short, X_long, y = X_short.to(device), X_long.to(device), y_short.to(device)

# ----- Step 4: Split Data and Create DataLoaders -----
split_idx = int(len(X_short) * 0.8)
X_short_train, X_short_test = X_short[:split_idx], X_short[split_idx:]
X_long_train, X_long_test = X_long[:split_idx], X_long[split_idx:]
y_train, y_test = y[:split_idx], y[split_idx:]

batch_size = 64
train_loader = DataLoader(TensorDataset(X_short_train, X_long_train, y_train), batch_size=batch_size, shuffle=True)
test_loader = DataLoader(TensorDataset(X_short_test, X_long_test, y_test), batch_size=batch_size, shuffle=False)

# ----- Step 5: Define the Dual LSTM Model -----
class DualLSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim=128, num_layers=2, dropout=0.3):
        super(DualLSTMModel, self).__init__()

        # Short-term LSTM
        self.lstm_short = nn.LSTM(input_dim, hidden_dim, num_layers=num_layers, batch_first=True, dropout=dropout)
        # Long-term LSTM
        self.lstm_long = nn.LSTM(input_dim, hidden_dim, num_layers=num_layers, batch_first=True, dropout=dropout)

        # Combine both
        self.fc = nn.Linear(hidden_dim * 2, 1)

    def forward(self, x_short, x_long):
        _, (hidden_short, _) = self.lstm_short(x_short)
        _, (hidden_long, _) = self.lstm_long(x_long)

        combined = torch.cat((hidden_short[-1], hidden_long[-1]), dim=1)
        output = self.fc(combined)
        return torch.sigmoid(output)

# Instantiate model
model = DualLSTMModel(input_dim=X_short.shape[2]).to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# ----- Step 6: Train the Model -----
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch_short, batch_long, batch_y in train_loader:
        optimizer.zero_grad()
        output = model(batch_short, batch_long).squeeze()
        loss = criterion(output, batch_y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss / len(train_loader):.4f}")

# ----- Step 7: Evaluate the Model -----
model.eval()
y_pred_list, y_test_list = [], []
with torch.no_grad():
    for batch_short, batch_long, batch_y in test_loader:
        output = model(batch_short, batch_long).squeeze()
        y_pred_list.extend(output.cpu().numpy())
        y_test_list.extend(batch_y.cpu().numpy())

# Using simple 0.5 threshold for now
y_pred_binary = (np.array(y_pred_list) > 0.5).astype(int)
y_test_cpu = np.array(y_test_list)

precision = precision_score(y_test_cpu, y_pred_binary)
recall = recall_score(y_test_cpu, y_pred_binary)
f1 = f1_score(y_test_cpu, y_pred_binary)
print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}")

# ----- Step 8: Predict Feeders Going Down for Each Week of January 2025 -----
weeks_to_predict = [
    pd.to_datetime("2025-01-06"),  # Week 1
    pd.to_datetime("2025-01-13"),  # Week 2
    pd.to_datetime("2025-01-20"),  # Week 3
    pd.to_datetime("2025-01-27"),  # Week 4
]

all_predictions = []

with torch.no_grad():
    for target_week in weeks_to_predict:
        df_filtered = df_weekly[df_weekly['Week'] < target_week]
        feeder_predictions = []
        feeders = df_filtered['fdr_Id'].unique()

        for feeder in feeders:
            feeder_df = df_filtered[df_filtered['fdr_Id'] == feeder].reset_index(drop=True)
            if len(feeder_df) >= max(seq_short, seq_long):
                seq_data_short = feeder_df[features].iloc[-seq_short:].values
                seq_data_long = feeder_df[features].iloc[-seq_long:].values

                seq_tensor_short = torch.tensor(seq_data_short, dtype=torch.float32).unsqueeze(0).to(device)
                seq_tensor_long = torch.tensor(seq_data_long, dtype=torch.float32).unsqueeze(0).to(device)

                pred_prob = model(seq_tensor_short, seq_tensor_long).item()
                feeder_predictions.append({
                    'week': target_week.strftime('%Y-%m-%d'),
                    'fdr_Id': feeder,
                    'predicted_probability': pred_prob
                })

        all_predictions.extend(feeder_predictions)

# Convert predictions to DataFrame and save
predictions_df = pd.DataFrame(all_predictions)
predictions_df.to_csv("/content/predicted_feeder_failures_jan2025.csv", index=False)
print("Predictions saved to /content/predicted_feeder_failures_jan2025.csv")


Epoch 1/10, Loss: 0.6412
Epoch 2/10, Loss: 0.6014
Epoch 3/10, Loss: 0.5816
Epoch 4/10, Loss: 0.5711
Epoch 5/10, Loss: 0.5734
Epoch 6/10, Loss: 0.5663
Epoch 7/10, Loss: 0.5644
Epoch 8/10, Loss: 0.5610
Epoch 9/10, Loss: 0.5609
Epoch 10/10, Loss: 0.5620
Precision: 0.5399, Recall: 0.3458, F1 Score: 0.4216
Predictions saved to /content/predicted_feeder_failures_jan2025.csv


# thresh opt 1 for all

In [78]:
# ----- Step 9: Compare Predictions with Actual Data for All Weeks (Global Threshold Optimization) -----

import numpy as np
import pandas as pd

# Load actual data
actual_data = pd.read_csv("/content/weekly_test.csv")
actual_data['Week'] = pd.to_datetime(actual_data['week'])

# Threshold search space
thresholds = np.arange(0.1, 0.9, 0.01)

all_week_predictions = []
all_week_actuals = []

for target_week in weeks_to_predict:
    week_str = target_week.strftime('%Y-%m-%d')

    week_predictions = predictions_df[predictions_df['week'] == week_str]
    week_actuals = actual_data[actual_data['Week'] == target_week]

    # Save all week data together
    all_week_predictions.append(week_predictions[['fdr_Id', 'predicted_probability']])
    all_week_actuals.append(week_actuals[['fdr_Id', 'Fault_Occurred']])

# Concatenate all weeks
all_week_predictions = pd.concat(all_week_predictions)
all_week_actuals = pd.concat(all_week_actuals)

# Merge predictions with actuals
merged = pd.merge(all_week_predictions, all_week_actuals, on='fdr_Id', how='inner')

# Now search for the best global threshold
best_threshold = 0.0
best_f1 = -1.0
best_precision = 0.0
best_recall = 0.0

for threshold in thresholds:
    merged['predicted_label'] = (merged['predicted_probability'] > threshold).astype(int)

    tp = ((merged['predicted_label'] == 1) & (merged['Fault_Occurred'] == 1)).sum()
    fp = ((merged['predicted_label'] == 1) & (merged['Fault_Occurred'] == 0)).sum()
    fn = ((merged['predicted_label'] == 0) & (merged['Fault_Occurred'] == 1)).sum()

    precision = tp / (tp + fp + 1e-9)
    recall = tp / (tp + fn + 1e-9)
    f1 = 2 * (precision * recall) / (precision + recall + 1e-9)

    if f1 > best_f1:
        best_f1 = f1
        best_threshold = threshold
        best_precision = precision
        best_recall = recall

print(f"\nBest Global Threshold Found: {best_threshold:.2f}")
print(f"Precision: {best_precision:.4f}, Recall: {best_recall:.4f}, F1 Score: {best_f1:.4f}")

# ---- Now Evaluate Week-by-Week Using This Best Threshold ----
evaluation_results = []

for target_week in weeks_to_predict:
    week_str = target_week.strftime('%Y-%m-%d')

    week_predictions = predictions_df[predictions_df['week'] == week_str]
    week_actuals = actual_data[actual_data['Week'] == target_week]

    predicted_failures = set(week_predictions[week_predictions['predicted_probability'] > best_threshold]['fdr_Id'])
    actual_failures = set(week_actuals[week_actuals['Fault_Occurred'] == 1]['fdr_Id'])

    true_positives = actual_failures & predicted_failures
    false_negatives = actual_failures - predicted_failures
    false_positives = predicted_failures - actual_failures

    precision = len(true_positives) / (len(true_positives) + len(false_positives) + 1e-9)
    recall = len(true_positives) / (len(true_positives) + len(false_negatives) + 1e-9)
    f1 = 2 * (precision * recall) / (precision + recall + 1e-9)

    evaluation_results.append({
        'week': week_str,
        'correct_predictions': len(true_positives),
        'missed_failures': len(false_negatives),
        'incorrect_predictions': len(false_positives),
        'precision': precision,
        'recall': recall,
        'f1_score': f1
    })

    print(f"\nResults for {week_str}:")
    print(f"Correct Predictions: {len(true_positives)}")
    print(f"Missed Failures: {len(false_negatives)}")
    print(f"Incorrect Predictions: {len(false_positives)}")
    print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}")

# Save evaluation results to CSV
evaluation_df = pd.DataFrame(evaluation_results)
evaluation_df.to_csv("/content/evaluation_results_jan2025.csv", index=False)
print("\nEvaluation results saved to /content/evaluation_results_jan2025.csv")

# ---- NEW PART: Calculate and print average F1 Score ----
average_f1 = evaluation_df['f1_score'].mean()
print(f"\nAverage F1 Score over all weeks: {average_f1:.4f}")



Best Global Threshold Found: 0.20
Precision: 0.5026, Recall: 0.7846, F1 Score: 0.6127

Results for 2025-01-06:
Correct Predictions: 42
Missed Failures: 14
Incorrect Predictions: 54
Precision: 0.4375, Recall: 0.7500, F1 Score: 0.5526

Results for 2025-01-13:
Correct Predictions: 47
Missed Failures: 16
Incorrect Predictions: 49
Precision: 0.4896, Recall: 0.7460, F1 Score: 0.5912

Results for 2025-01-20:
Correct Predictions: 54
Missed Failures: 13
Incorrect Predictions: 42
Precision: 0.5625, Recall: 0.8060, F1 Score: 0.6626

Results for 2025-01-27:
Correct Predictions: 50
Missed Failures: 10
Incorrect Predictions: 46
Precision: 0.5208, Recall: 0.8333, F1 Score: 0.6410

Evaluation results saved to /content/evaluation_results_jan2025.csv

Average F1 Score over all weeks: 0.6119


# just to test threshold out

In [67]:
# ----- Step 9: Compare Predictions with Actual Data for Each Week -----
actual_data = pd.read_csv("/content/weekly_test.csv")
actual_data['Week'] = pd.to_datetime(actual_data['week'])

evaluation_results = []
threshold = 0.31  # Use the same threshold

for target_week in weeks_to_predict:
    week_str = target_week.strftime('%Y-%m-%d')

    # Filter predictions and actuals for this week
    week_predictions = predictions_df[predictions_df['week'] == week_str]
    predicted_failures = set(week_predictions[week_predictions['predicted_probability'] > threshold]['fdr_Id'])

    week_actuals = actual_data[actual_data['Week'] == target_week]
    actual_failures = set(week_actuals[week_actuals['Fault_Occurred'] == 1]['fdr_Id'])

    true_positives = actual_failures & predicted_failures
    false_negatives = actual_failures - predicted_failures
    false_positives = predicted_failures - actual_failures

    precision = len(true_positives) / (len(true_positives) + len(false_positives) + 1e-9)
    recall = len(true_positives) / (len(true_positives) + len(false_negatives) + 1e-9)
    f1 = 2 * (precision * recall) / (precision + recall + 1e-9)

    evaluation_results.append({
        'week': week_str,
        'correct_predictions': len(true_positives),
        'missed_failures': len(false_negatives),
        'incorrect_predictions': len(false_positives),
        'precision': precision,
        'recall': recall,
        'f1_score': f1
    })

    print(f"\nResults for {week_str}:")
    print(f"Correct Predictions: {len(true_positives)}")
    print(f"Missed Failures: {len(false_negatives)}")
    print(f"Incorrect Predictions: {len(false_positives)}")
    print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}")

# Save evaluation results to CSV
evaluation_df = pd.DataFrame(evaluation_results)
evaluation_df.to_csv("/content/evaluation_results_jan2025.csv", index=False)
print("\nEvaluation results saved to /content/evaluation_results_jan2025.csv")

# ---- NEW PART: Calculate and print average F1 Score ----
average_f1 = evaluation_df['f1_score'].mean()
print(f"\nAverage F1 Score over all weeks: {average_f1:.4f}")



Results for 2025-01-06:
Correct Predictions: 42
Missed Failures: 14
Incorrect Predictions: 46
Precision: 0.4773, Recall: 0.7500, F1 Score: 0.5833

Results for 2025-01-13:
Correct Predictions: 47
Missed Failures: 16
Incorrect Predictions: 41
Precision: 0.5341, Recall: 0.7460, F1 Score: 0.6225

Results for 2025-01-20:
Correct Predictions: 55
Missed Failures: 12
Incorrect Predictions: 33
Precision: 0.6250, Recall: 0.8209, F1 Score: 0.7097

Results for 2025-01-27:
Correct Predictions: 48
Missed Failures: 12
Incorrect Predictions: 40
Precision: 0.5455, Recall: 0.8000, F1 Score: 0.6486

Evaluation results saved to /content/evaluation_results_jan2025.csv

Average F1 Score over all weeks: 0.6410


#thresholds individual for each weeks

In [68]:
# ----- Step 9: Compare Predictions with Actual Data for Each Week with Threshold Optimization -----

import numpy as np
import pandas as pd

# Load actual data
actual_data = pd.read_csv("/content/weekly_test.csv")
actual_data['Week'] = pd.to_datetime(actual_data['week'])

evaluation_results = []

# Define threshold search space
thresholds = np.arange(0.1, 0.9, 0.01)

for target_week in weeks_to_predict:
    week_str = target_week.strftime('%Y-%m-%d')

    # Filter predictions and actuals for this week
    week_predictions = predictions_df[predictions_df['week'] == week_str]
    week_actuals = actual_data[actual_data['Week'] == target_week]

    best_threshold = 0.0
    best_f1 = -1.0
    best_precision = 0.0
    best_recall = 0.0
    best_tp = best_fn = best_fp = 0

    # Optimize threshold for best F1
    for threshold in thresholds:
        predicted_failures = set(week_predictions[week_predictions['predicted_probability'] > threshold]['fdr_Id'])
        actual_failures = set(week_actuals[week_actuals['Fault_Occurred'] == 1]['fdr_Id'])

        true_positives = actual_failures & predicted_failures
        false_negatives = actual_failures - predicted_failures
        false_positives = predicted_failures - actual_failures

        precision = len(true_positives) / (len(true_positives) + len(false_positives) + 1e-9)
        recall = len(true_positives) / (len(true_positives) + len(false_negatives) + 1e-9)
        f1 = 2 * (precision * recall) / (precision + recall + 1e-9)

        if f1 > best_f1:
            best_f1 = f1
            best_threshold = threshold
            best_precision = precision
            best_recall = recall
            best_tp = len(true_positives)
            best_fn = len(false_negatives)
            best_fp = len(false_positives)

    evaluation_results.append({
        'week': week_str,
        'best_threshold': best_threshold,
        'correct_predictions': best_tp,
        'missed_failures': best_fn,
        'incorrect_predictions': best_fp,
        'precision': best_precision,
        'recall': best_recall,
        'f1_score': best_f1
    })

    print(f"\nResults for {week_str}:")
    print(f"Best Threshold: {best_threshold:.2f}")
    print(f"Correct Predictions: {best_tp}")
    print(f"Missed Failures: {best_fn}")
    print(f"Incorrect Predictions: {best_fp}")
    print(f"Precision: {best_precision:.4f}, Recall: {best_recall:.4f}, F1 Score: {best_f1:.4f}")

# Save evaluation results to CSV
evaluation_df = pd.DataFrame(evaluation_results)
evaluation_df.to_csv("/content/evaluation_results_jan2025.csv", index=False)
print("\nEvaluation results saved to /content/evaluation_results_jan2025.csv")

# ---- NEW PART: Calculate and print average F1 Score ----
average_f1 = evaluation_df['f1_score'].mean()
print(f"\nAverage F1 Score over all weeks: {average_f1:.4f}")



Results for 2025-01-06:
Best Threshold: 0.19
Correct Predictions: 49
Missed Failures: 7
Incorrect Predictions: 57
Precision: 0.4623, Recall: 0.8750, F1 Score: 0.6049

Results for 2025-01-13:
Best Threshold: 0.19
Correct Predictions: 53
Missed Failures: 10
Incorrect Predictions: 53
Precision: 0.5000, Recall: 0.8413, F1 Score: 0.6272

Results for 2025-01-20:
Best Threshold: 0.33
Correct Predictions: 53
Missed Failures: 14
Incorrect Predictions: 27
Precision: 0.6625, Recall: 0.7910, F1 Score: 0.7211

Results for 2025-01-27:
Best Threshold: 0.41
Correct Predictions: 40
Missed Failures: 20
Incorrect Predictions: 18
Precision: 0.6897, Recall: 0.6667, F1 Score: 0.6780

Evaluation results saved to /content/evaluation_results_jan2025.csv

Average F1 Score over all weeks: 0.6578
